In [1]:
import numpy as np
import torch as t
from torch.distributions import Normal, Categorical, Bernoulli
from torch.distributions import MultivariateNormal as MvNormal
import matplotlib.pyplot as plt
%matplotlib widget
from ipywidgets import FloatSlider, IntSlider, interact, interact_manual

$$
\newcommand{\bracket}[3]{\left#1 #3 \right#2}
\newcommand{\b}{\bracket{(}{)}}
\newcommand{\Bernoulli}{{\rm Bernoulli}\b}
\newcommand{\Categorical}{{\rm Categorical}\b}
\newcommand{\x}{\mathbf{x}}
\newcommand{\X}{\mathbf{X}}
\newcommand{\m}{\boldsymbol{\mu}}
\newcommand{\P}{{\rm P}\b}
\newcommand{\dd}[2][]{\frac{\partial #1}{\partial #2}}
\newcommand{\S}{\mathbf{\Sigma}}
\newcommand{\Sh}{\mathbf{\hat{\Sigma}}}
\newcommand{\mh}{\boldsymbol{\hat{\mu}}}
\newcommand{\N}{\mathcal{N}\b}
\newcommand{\det}{\bracket{\lvert}{\rvert}}
\newcommand{\sb}{\bracket{[}{]}}
\newcommand{\E}{\mathbb{E}\sb}
\newcommand{\Var}{{\rm Var}\sb}
\newcommand{\Cov}{{\rm Cov}\sb}
\DeclareMathOperator*{\argmax}{arg\,max}
\DeclareMathOperator*{\argmin}{arg\,min}
\newcommand{\ph}{\hat{p}}
\newcommand{\at}{\bracket{.}{\rvert}}
\newcommand{\w}{\mathbf{w}}
\newcommand{\W}{\mathbf{W}}
\newcommand{\W}{\mathbf{W}}
\newcommand{\Wh}{\mathbf{\hat{W}}}
\newcommand{\Y}{\mathbf{Y}}
\newcommand{\L}{\mathcal{L}}
\newcommand{\wh}{\mathbf{\hat{w}}}
\newcommand{\y}{\mathbf{y}}
\newcommand{\0}{\mathbf{0}}
\newcommand{\I}{\mathbf{I}}
\newcommand{\La}{\mathbf{\Lambda}}
\newcommand{\S}{\mathbf{\Sigma}}
\newcommand{\Sprior}{\S_\text{prior}}
\newcommand{\Spost}{\S_\text{post}}
\newcommand{\mprior}{\m_\text{prior}}
\newcommand{\mpost}{\m_\text{post}}
\newcommand{\Xt}{\tilde{\X}}
\newcommand{\yt}{\tilde{\y}}
\newcommand{\p}{\mathbf{p}}
\newcommand{\l}{\boldsymbol{\ell}}
\DeclareMathOperator*{\softmax}{softmax}
\DeclareMathOperator*{\sigmoid}{sigmoid}
$$

<h1> Lecture 4: Classification </h1>

Classification is almost exactly the same as regression, except that:
<ul>
    <li> The outputs, $y$, are discrete class-labels. </li>
    <li> Almost all interesting/useful algorithms require iterative solutions </li>
</ul>
    
The same considerations are relevant, including,
<ul>
    <li> Overfitting </li>
    <li> Regularisation </li>
    <li> Cross-validation </li>
    <li> Bayes (but this is much harder, as there aren't any exact solutions) </li>
</ul>

<h2> Prerequisites: Bernoulli distribution for two-class classification</h2>
Samples from the Bernoulli distribution are either $0$ or $1$, with probability given by the parameter,

\begin{align}
  \P{y| p} &= \Bernoulli{y; p} = y p + (1-y) (1-p)
\end{align}

In [3]:
Py = Bernoulli(probs=0.8)
print(Py.sample((10,)))
print(Py.log_prob(t.tensor([0., 1.])).exp())

tensor([1., 1., 0., 0., 1., 1., 0., 1., 0., 0.])
tensor([0.2000, 0.8000])


<h3> Logit parameterisation of the Bernoulli distribution and the sigmoid </h3>

Working directly with the probabilities turns out to be problematic:
<ul>
    <li> Probabilities live in a strange range, $0 \leq p \leq 1$. </li>
    <li> There is a strong risk of numerical underflow, which breaks algorithms such as (stochastic) gradient descent. </li>
</ul>

Instead, we can also treat the Bernoulli parameter as a logits vector, $\ell$, defined such that,

\begin{align}
  p &= \sigmoid(\ell) = \sigma(\ell)\\
  p &= \frac{1}{1+e^{-\ell}}
\end{align}

Now, no matter what $\l$ is, the probabilities must lie in the right range, and they are much less likely to underflow.

PyTorch allows you to directly use the logit parameterisation,

In [4]:
ls = t.linspace(-4, 4, 100)
ps = t.sigmoid(ls)
fig, ax = plt.subplots()
ax.set_xlabel("$\ell$")
ax.set_ylabel("$p = \sigma(\ell)$")
ax.plot(ls, ps);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
Py = Bernoulli(logits=-2)
print(Py.sample((10,)))
print(Py.log_prob(t.tensor([0., 1.])).exp())

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor([0.8808, 0.1192])


<h2> Prerequisites: Categorical distribution for multi-class classification </h2>

The Bernoulli distribution takes one parameter and gives the probability of two classes.  But what about multiple classes?

Samples from the Categorical distribution are integers $0 \leq y < K$, with probability given explicity by a length $K$ vector of, $\p$,

\begin{align}
  \P{y| \p} &= \Categorical{y; \p} = p_y
\end{align}

In [7]:
#A uniform Categorical distribution,
Py = Categorical(probs=t.ones(10)/10)
print(Py.sample((20,)))

tensor([0, 2, 7, 1, 4, 2, 8, 4, 5, 0, 9, 8, 3, 1, 5, 9, 4, 1, 3, 9])


In [8]:
ys = t.arange(10)
print(ys)
Py.log_prob(ys).exp()

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
        0.1000])

In [10]:
#A non-uniform Categorical distribution,
p = t.tensor([0.5, 0.1, 0.1, 0.1, 0.1, 0.1])
Py = Categorical(probs=p)
y = Py.sample((20,))
y

tensor([0, 0, 5, 2, 2, 3, 4, 5, 1, 1, 1, 4, 0, 2, 5, 1, 4, 1, 5, 3])

In [20]:
Py.log_prob(t.arange(6)).exp()

tensor([0.5000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000])

<h3> Logit parameterisation of the Categorical distribution and the softmax </h3>

Working directly with the probabilities turns out to be problematic:
<ul>
    <li> Probabilities live in a strange range, $0 \leq p_i \leq 1$. </li>
    <li> Probabilities must sum to $1$. </li>
    <li> There is a strong risk of numerical underflow, which breaks algorithms such as (stochastic) gradient descent. </li>
</ul>

Instead, we can also treat the Categorical parameter as a logits vector, $\l$, defined such that,

\begin{align}
  \p &= \softmax(\l)\\
  p_i &= \frac{e^{\ell_i}}{\sum_{j=0}^{K-1} e^{\ell_j}}
\end{align}

Now, no matter what $\l$ is, the probabilities must lie in the right range, they must normalize, and they are much less likely to underflow.

PyTorch allows you to directly use the logit parameterisation,

In [12]:
# Uniform Categorical
Py = Categorical(logits = t.ones(10))
print(Py.sample((20,)))
print(Py.log_prob(t.arange(10)).exp())

tensor([4, 8, 9, 2, 2, 9, 2, 2, 3, 7, 5, 9, 1, 7, 6, 0, 9, 6, 7, 7])
tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
        0.1000])


In [13]:
# Non-uniform Categorical
l = t.tensor([1., 0., -1., -2., -10.])
Py = Categorical(logits = l)
print(Py.sample((20,)))
print(Py.log_prob(t.arange(5)).exp())

tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0])
tensor([6.4391e-01, 2.3688e-01, 8.7143e-02, 3.2058e-02, 1.0754e-05])


<h2> Formalising maximum-likelihood supervised learning </h2>

For linear regression, the goal was to predict the distribution over a floating-point $y_\lambda$ based on a given value for $\x_\lambda$,

\begin{align}
  \P{y_\lambda| \x_\lambda} &= \N{y_\lambda; \x_\lambda \cdot \w, \sigma^2}
\end{align}

In classification, the goal is similar: we again want to predict a distribution over $y_\lambda$, based on an input, $\x_\lambda$.  The only difference is that here, $y_\lambda$ is an integer from $0$ to $K-1$.  Thus, when we predict a probability over $y_\lambda$ it either needs to be Bernoulli (for two classes), or categorical (if there can be more classes,

\begin{align}
  \P{y_\lambda| \x_\lambda} &= \Bernoulli{y_\lambda; \sigma\b{\x_\lambda \cdot \w}}\\
  \P{y_\lambda| \x_\lambda} &= \Categorical{y_\lambda; \softmax\b{\x_\lambda \W}}
\end{align}

As an example, consider the "Iris" dataset, which is about classifying flowers based on features such as Petal length.

In [25]:
from sklearn import datasets
iris = datasets.load_iris()
_X = t.tensor(iris['data'][50:, [0, 2]]).float()
_Y = (t.tensor(iris['target'][50:])-1).float()

fig, ax = plt.subplots()
ax.set_xlabel("$x_{\lambda 0}$")
ax.set_ylabel("$x_{\lambda 1}$")
ax.scatter(_X[:, 0], _X[:, 1], c=_Y);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Split the data into train and test (we shuffle the data first, because the classes are ordered)

In [26]:
t.manual_seed(0)
perm = t.randperm(_X.shape[0])
X = _X[perm, :]
X = t.cat([X, t.ones(X.shape[0], 1)], -1)
Y = _Y[perm][:, None]

X_train = X[:70, :]
Y_train = Y[:70, :]
X_test  = X[70:, :]
Y_test  = Y[70:, :]

There are some more quicker iterative algorithms.  But they don't add much understanding.  So instead, we use PyTorch-magic to do gradient-descent.

In [27]:
W = t.randn((3,1), requires_grad=True)/100

for i in range(50000):
    L = Bernoulli(logits=X_train@W).log_prob(Y_train).sum()
    
    dW = t.autograd.grad(outputs=L, inputs=(W,))[0]
    if 0==i % 1000:
        print(L.item())
    W.data += 0.001*dW

-48.974510192871094
-20.449899673461914
-17.061647415161133
-15.644773483276367
-14.787422180175781
-14.168648719787598
-13.676431655883789
-13.261651992797852
-12.899398803710938
-12.575631141662598
-12.281757354736328
-12.01214599609375
-11.762899398803711
-11.53117561340332
-11.31480884552002
-11.112078666687012
-10.92162799835205
-10.742270469665527
-10.573012351989746
-10.413020133972168
-10.261512756347656
-10.117838859558105
-9.981404304504395
-9.851656913757324
-9.728116989135742
-9.610358238220215
-9.49797534942627
-9.39061450958252
-9.287946701049805
-9.189657211303711
-9.09546184539795
-9.005102157592773
-8.918354988098145
-8.835001945495605
-8.754850387573242
-8.677701950073242
-8.603403091430664
-8.53178882598877
-8.46267032623291
-8.395984649658203
-8.331559181213379
-8.269283294677734
-8.20904541015625
-8.150764465332031
-8.094314575195312
-8.039603233337402
-7.9865803718566895
-7.935168266296387
-7.88528299331665
-7.836832046508789


Now, we can plot the classification boundary,

In [30]:
fig, ax = plt.subplots()
ax.set_xlabel("$x_{\lambda 0}$")
ax.set_ylabel("$x_{\lambda 1}$")
ax.scatter(_X[:, 0], _X[:, 1], c=_Y);

x0range = t.linspace(*ax.get_xlim(), 500)
x1range = t.linspace(*ax.get_ylim(), 500)

x0s, x1s = t.meshgrid(x0range, x1range)
xs = t.stack([x0s, x1s, t.ones(500, 500)], -1)

ps = Bernoulli(logits=xs@W.detach()).probs[:, :, 0]
ax.contour(x0s, x1s, 0.5<ps);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Now, we can compute the classification-error,

In [31]:
def class_error(X, Y):
    Py = Bernoulli(logits=X@W)
    pred = 0.5 < Py.probs
    N_correct = (pred == Y).sum()
    print(f"{N_correct}/{X.shape[0]} = {100.*N_correct/X.shape[0]}%")
    
print("Training correct")
class_error(X_train, Y_train)

print("Test correct")
class_error(X_test, Y_test)

Training correct
67/70 = 95.71428680419922%
Test correct
28/30 = 93.33333587646484%


Note that the classifier performs really well on training data, but less well on test data.

Indicates evidence of overfitting.

<h2> Overfitting in classification</h2>

In [33]:
def cheb(xs, c):
    # c is int
    coefs = c*[0] + [1]
    return np.polynomial.chebyshev.chebval(xs, coefs)
def chebX(X, order):
    assert (-1 <= X).all() and (X <= 1).all()
    
    xs = []
    for c in range(order):
        xs.append(cheb(X, c))
    return t.cat(xs, 1)
t.manual_seed(0)
N = 100
X = 2*t.rand(N, 1)-1
W_true = t.tensor([[4.]])
Y = Bernoulli(logits=X@W_true).sample()

def plot(order):
    Xe= chebX(X, order)
    W = t.zeros((order, 1), requires_grad=True)

    for i in range(15000):
        L = Bernoulli(logits=Xe@W).log_prob(Y).sum()
        if 0==i % 2000:
            print(L.item())
        dW = t.autograd.grad(outputs=L, inputs=(W,))[0]
        W.data += 0.001*dW
    
    fig, ax = plt.subplots()
    ax.set_xlabel("$x$")
    ax.set_ylabel("probability / $y$")
    ax.scatter(X, Y)
    xs = t.linspace(-1, 1, 100)[:, None]
    ps_fitted = Bernoulli(logits=chebX(xs, order)@W).probs.detach()
    ps_true = Bernoulli(logits=xs@W_true).probs.detach()
    ax.plot(xs, ps_fitted, label="fitted probability")
    ax.plot(xs, ps_true, label="true probability")
    ax.legend()
    
interact_manual(plot, order=IntSlider(min=2, max=30));

interactive(children=(IntSlider(value=2, description='order', max=30, min=2), Button(description='Run Interact…

You can use the same techniques to control overfitting:
<ul>
    <li> Regularisation </li>
    <li> Cross-validation (either on the number of classification errors for test points, or on the test-log-likelihood </li>
    <li> Bayesian inference (though its much harder here because the posterior over the weights doesn't have an analytiv form)
<ul>

<h2> Simpler, heuristic methods for classification </h2>

We've seen a bunch of complex methods, and implementing them is often quite involved.  Is there anything simpler?

The answer is yes!

<h3> K-nearest neighbour </h3>

One approach is to look at the nearby datapoints.  If the nearby points come from one class, then the chances are that our datapoint come from the same class.

In [44]:
from sklearn import datasets
iris = datasets.load_iris()
_X = t.tensor(iris['data'][50:, [0, 2]]).float()
_Y = (t.tensor(iris['target'][50:])-1)

t.manual_seed(0)
perm = t.randperm(_X.shape[0])
X = _X[perm, :]
Y = _Y[perm]


def plot(K):
    fig, ax = plt.subplots()
    ax.set_xlabel("$x_{\lambda 0}$")
    ax.set_ylabel("$x_{\lambda 1}$")
    ax.scatter(_X[:, 0], _X[:, 1], c=_Y);

    x0range = t.linspace(*ax.get_xlim(), 500)
    x1range = t.linspace(*ax.get_ylim(), 500)

    x0s, x1s = t.meshgrid(x0range, x1range)
    xs = t.stack([x0s, x1s], -1)

    X_exp = X[:, None, None, :]
    
    dist2 = ((X_exp - xs)**2).sum(-1)
    elems = (-dist2).topk(K, dim=0).indices
    pred = (0.5<Y[elems].float().mean(0)).float()
    ax.contour(x0s, x1s, pred)
    
interact_manual(plot, K=IntSlider(min=1, max=11, step=2));

interactive(children=(IntSlider(value=1, description='K', max=11, min=1, step=2), Button(description='Run Inte…

<h3> Cross-validation </h3>

Choose $K$ using cross-validation.  Note that leave-one-out cross validation is very suitable here, as we make a prediction for each point, based on its neighbours, in parallel.

<h3> Weighted-nearest neighbour </h3>

This approach is to take a weighted average of nearby datapoints,

\begin{align}
  p_{\lambda, y} &= \frac{\sum_{\lambda'}k(\x_\lambda, \x_{\lambda'}) \delta_{y_\lambda, y_{\lambda'}}}{\sum_{\lambda''} k(\x_\lambda, \x_{\lambda''})}
\end{align}

where we might use a squared-exponential kernel/weights (but many other choices are available),

\begin{align}
  k(\x_\lambda, \x_{\lambda'}) = e^{-(\x_\lambda - \x_{\lambda'})^2/(2 b)}
\end{align}

where $b$ is a bandwidth/lengthscale parameter.

In [50]:
from sklearn import datasets
iris = datasets.load_iris()
_X = t.tensor(iris['data'][50:, [0, 2]]).float()
_Y = (t.tensor(iris['target'][50:])-1)

t.manual_seed(0)
perm = t.randperm(_X.shape[0])
X = _X[perm, :]
Y = _Y[perm]

def plot(bandwidth):
    fig, ax = plt.subplots()
    ax.set_xlabel("$x_{\lambda 0}$")
    ax.set_ylabel("$x_{\lambda 1}$")
    ax.scatter(_X[:, 0], _X[:, 1], c=_Y);

    x0range = t.linspace(*ax.get_xlim(), 500)
    x1range = t.linspace(*ax.get_ylim(), 500)

    x0s, x1s = t.meshgrid(x0range, x1range)
    xs = t.stack([x0s, x1s], -1)

    X_exp = X[:, None, None, :]

    dist2 = ((X_exp - xs)**2).sum(-1)
    ws = t.exp(-dist2/(2*bandwidth**2))
    pred = (ws*Y_train[:, None, None]).sum(0) / ws.sum(0)

    ax.contour(x0s, x1s, 0.5<pred);
    
interact_manual(plot, bandwidth=FloatSlider(min=0.15, max=1., step=0.01));

interactive(children=(FloatSlider(value=0.15, description='bandwidth', max=1.0, min=0.15, step=0.01), Button(d…

<h3> Cross-validation </h3>

Choose bandwidth using cross-validation.  Note that leave-one-out cross validation is very suitable here, as we make a prediction for each point, based on its neighbours, in parallel.

<h3> Nearest centroids </h3>

<h2> Using Bayes theorem to do classification </h2>

Above, we directly learned parameters that map from the data point, to the class.  Critically, though, we never write down a distribution over data-points, $\P{x_\lambda| y_\lambda}$, so we never have a way to generate something that looks like the data.

However, we could do this.  In particular, for each class, we could separately learn $\P{x_\lambda| y_\lambda}$, then use Bayes theorem to give us the probability of a class, conditioned on a data-point,

\begin{align}
  \P{y_\lambda| \x_\lambda} &\propto \P{y_\lambda} \P{\x_\lambda| y_\lambda}.
\end{align}

The simplest example is to take,

\begin{align}
  \P{\x_\lambda| y_\lambda} &= \N{\x_\lambda; \m_{y_\lambda}, \sigma^2 \I}.
\end{align}

Critically, this distribution is easy to fit (in comparison to _clustering_, which we will look at later), because $y_\lambda$ is known.

In [27]:
from sklearn import datasets
iris = datasets.load_iris()
X = t.tensor(iris['data'][:100, [0, 2]]).float()
Y = t.tensor(iris['target'][:100])

X0s = X[Y==0, :]
X1s = X[Y==1, :]

m0 = X0s.mean(0)
m1 = X1s.mean(0)

fig, ax = plt.subplots()
ax.set_xlabel("$x_{\lambda 0}$")
ax.set_ylabel("$x_{\lambda 1}$")
ax.scatter(X[:, 0], X[:, 1], c=Y)
ax.scatter(
    np.array([m0[0], m1[0]]), 
    np.array([m0[1], m1[1]]),
    c=np.array([0,1]),
    s=300
)

/Users/laurence/programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if sys.path[0] == '':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
Ns = [
    MvNormal(m0, 0.3*t.eye(2)),
    MvNormal(m1, 0.3*t.eye(2))
]

x0range = t.linspace(*ax.get_xlim(), 100)
x1range = t.linspace(*ax.get_ylim(), 100)

x0s, x1s = t.meshgrid(x0range, x1range)
xs = t.stack([x0s, x1s], -1)

def prob(y):
    fig, ax = plt.subplots()
    ax.set_xlabel("$x_{\lambda 0}$")
    ax.set_ylabel("$x_{\lambda 1}$")
    ax.scatter(X[:, 0], X[:, 1], c=Y)
    p = Ns[y].log_prob(xs).exp()
    ax.contour(x0s, x1s, p)

interact_manual(prob, y=IntSlider(min=0, max=1));

interactive(children=(IntSlider(value=0, description='y', max=1), Button(description='Run Interact', style=But…

To select the class, we use,

\begin{align}
  \P{y_\lambda| \x_\lambda} &\propto \P{y_\lambda} \P{\x_\lambda| y_\lambda}
\end{align}

We take $\P{y_\lambda}$ to be a uniform Bernoulli/Categorical, so this distribution is,

\begin{align}
  \log \P{y_\lambda| \x_\lambda} &= \log \P{\x_\lambda| y_\lambda} + \text{const}\\
  \log \P{y_\lambda| \x_\lambda} &= -\tfrac{1}{2 \sigma^2} \b{\x_\lambda - \m_{y_\lambda}}^2 + \text{const}\\
\end{align}

So finding the $y_\lambda$ with the maximum posterior probability is equivalent to taking the closest "centroid" (i.e. $\m_{y_\lambda}$).

In [72]:
fig, ax = plt.subplots()
ax.set_xlabel("$x_{\lambda 0}$")
ax.set_ylabel("$x_{\lambda 1}$")
ax.scatter(X[:, 0], X[:, 1], c=Y)
unnorm_p0s = Ns[0].log_prob(xs).exp()
unnorm_p1s = Ns[1].log_prob(xs).exp()
norm_p1 = unnorm_p1s/(unnorm_p0s + unnorm_p1s)

ax.contour(x0s, x1s, norm_p1);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …